# Import library

In [260]:
import cv2
import os
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from keras.utils import normalize
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Reading dataset

In [261]:
no_dir = os.listdir('./datasets/sort_crop_no/')
yes_dir = os.listdir('./datasets/sort_crop_yes/')

In [262]:
len(no_dir)

1185

In [263]:
len(yes_dir)

967

In [264]:
dataset,label = [],[]
for i,cur_img_dir in enumerate(no_dir):
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./datasets/sort_crop_no/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        dataset.append(np.array(img))
        label.append(0)

In [265]:
for i,cur_img_dir in enumerate(yes_dir):
    #check type of image
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./datasets/sort_crop_yes/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        dataset.append(np.array(img))
        label.append(1)

In [266]:
dataset = np.array(dataset)
label = np.array(label)
dataset.shape

(2152, 64, 64, 3)

In [267]:
label.shape

(2152,)

In [268]:
print(f'yes observe:{sum(label)}, no observe:{len(label)-sum(label)}')

yes observe:967, no observe:1185


# Split and normalize data


## 1. Split


In [269]:
np.array(dataset)  
label = np.array(label)  

x_train, x_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.2 )        

x_train = np.array(x_train, dtype=float)
x_test= np.array(x_test, dtype=float)

x_train_normalized=x_train/255
x_test_normalized=x_test/255

y_train=to_categorical(y_train, num_classes=2)
y_test=to_categorical(y_test, num_classes=2)

In [270]:
print(f'X train shape: {x_train.shape}\nY train shape: {y_train.shape}\nX test shape: {x_test.shape}\nY test shape: {y_test.shape}\nX validation shape: {x_val.shape}\nY validation shape: {x_val.shape}')

X train shape: (1721, 64, 64, 3)
Y train shape: (1721, 2)
X test shape: (431, 64, 64, 3)
Y test shape: (431, 2)
X validation shape: (431, 64, 64, 3)
Y validation shape: (431, 64, 64, 3)


# Building Model

In [271]:
from keras.models import Sequential 
from keras.layers import (
    Conv2D,
    MaxPooling2D,
    Activation,
    Dropout,
    Flatten,
    Dense
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, MaxPooling2D, Dropout
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, MobileNet, DenseNet121
from tensorflow.keras.models import Model
import numpy as np

In [272]:
from keras.models import load_model
def create_original_model():
    model=load_model('Auto_Braintumor10EpochsCategorical.h5')
    return model

# Try other network to choose the best model


## LeNet-5

In [273]:

INPUT_SIZE = 64

def create_lenet5():
    model = Sequential([
        Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(INPUT_SIZE, INPUT_SIZE, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(16, kernel_size=(5, 5), activation='sigmoid'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(120, activation='sigmoid'),
        Dense(84, activation='sigmoid'),
        Dense(2, activation='softmax')
    ])
    return model


## AlexNet

In [274]:

INPUT_SIZE = 64

def create_alexnet():
    model = Sequential([
        Conv2D(96, kernel_size=(11, 11), strides=4, activation='relu', input_shape=(INPUT_SIZE, INPUT_SIZE, 3)),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        Conv2D(256, kernel_size=(5, 5), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'),
        Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'),
        Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.02),
        Dense(4096, activation='relu'),
        Dropout(0.02),
        Dense(2, activation='softmax')
    ])
    return model


## MobileNet

In [275]:

INPUT_SIZE = 64

def create_mobilenet():
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(INPUT_SIZE, INPUT_SIZE, 3))
    
    x = base_model.output
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model


### Compile and train models

In [276]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def compile_and_train(model, x_train_normalized, y_train, x_test_normalized, y_test):
    model.compile(
        loss='categorical_crossentropy',  
        optimizer='adam',  
        metrics=['accuracy']  
    )
    
    early_stopping = EarlyStopping(
        monitor='val_loss',  
        patience=5,  
        restore_best_weights=True  
    )
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',  
        factor=0.1,  
        patience=3,  
        min_lr=1e-6  
    )
    

    history = model.fit(
        x_train_normalized, 
        y_train, 
        batch_size=16, 
        epochs=2,  
        validation_data=(x_test_normalized, y_test),  
        shuffle=True,  
        callbacks=[early_stopping, reduce_lr]  
    )


    loss, accuracy = model.evaluate(x_test_normalized, y_test, verbose=1)

    return accuracy, history


# Training model

In [277]:
models = [
    ('Original', create_original_model()),
    ('LeNet-5', create_lenet5()),
    ('AlexNet', create_alexnet()),
    ('MobileNet', create_mobilenet()),
]
accuracies = []


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13904\1564747537.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(INPUT_SIZE, INPUT_SIZE, 3))


In [278]:
for name, model in models:
    accuracy = compile_and_train(model, x_train_normalized, y_train, x_test_normalized, y_test)
    accuracies.append((name, accuracy))

Epoch 1/2
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9656 - loss: 0.0913 - val_accuracy: 0.9304 - val_loss: 0.1839 - learning_rate: 0.0010
Epoch 2/2
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9687 - loss: 0.0727 - val_accuracy: 0.9722 - val_loss: 0.1001 - learning_rate: 0.0010
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9682 - loss: 0.1093
Epoch 1/2
108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5295 - loss: 0.7048 - val_accuracy: 0.5452 - val_loss: 0.6913 - learning_rate: 0.0010
Epoch 2/2
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5583 - loss: 0.6880 - val_accuracy: 0.4548 - val_loss: 0.7026 - learning_rate: 0.0010
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5828 - loss: 0.6898 
Epoch 1/2
108/108 ━━━━━━━━━━━━━━━━━━━━ 33s 290ms/step - accuracy: 0.5300 - loss: 1.0872 - val_accuracy: 0.5452 - val_loss: 0.6872 - learning_rate: 0.0010
Epoch 2/2
108/108 ━━━━━━━━━━━━━━━━━━━━ 31s 284ms/step - accuracy: 0.5711 - loss: 0.6799 - val

# Print accuracy of all model

In [283]:
accuracies.sort(key=lambda x: x[1], reverse=True)
    
# Print models and their accuracies
print("List of models sorted by accuracy:")
for name, accuracy in accuracies:
    print(f'Model: {name}, Accuracy: {accuracy:.4f}')
    
# Print the best model
best_model_name, best_accuracy = accuracies[0]
print(f'Best Model: {best_model_name}, Accuracy: {best_accuracy:.4f}')

TypeError: '<' not supported between instances of 'History' and 'History'

**So, the original model is the best!**